#### Precipitation and Air Temperature data were downloaded from
- NASA's [Daymet](https://daymet.ornl.gov/) using coordinates from the Clemson University Pee Dee Research Center
- latitude: 34.289	
- longitude: -79.737

#### Wind and humidity data were downloaded from
- Climate [Engine](https://clim-engine.appspot.com/climateEngine) using coordinates from the Clemson Pee Dee Research and Education Center
- latitude: 34.289	
- longitude: -79.737

#### These are more precise environmental data which will be used in combination with the trait [data](https://github.com/MagicMilly/for-data-publication/tree/main/data/processed) from Brenton et al. 2016 at the Clemson University Pee Dee Research Center

In [3]:
import datetime
import numpy as np
import pandas as pd

### A. Daymet Download

In [4]:
daymet_0 = pd.read_csv('data/raw/clemson_raw_daily_2014_updated.csv')
print(daymet_0.shape)
# daymet_0.head()

(365, 9)


### Columns from Daymet Download
- year
- day of year
- dayl - daylength, seconds per day
- precipitation in mm
- srad - shortwave radiation (can be used with dayl to calculate total daily radiation)
- swe - snow water equivalent
- air temperature max in C
- air temperature min in C
- vp - daily average partial pressure of water vapor, in Pascals

#### Add Dates

In [5]:
date_range_2014 = pd.date_range(start='2014-01-01', end='2014-12-31')

In [6]:
daymet_1 = daymet_0.copy()
daymet_1['date'] = date_range_2014
print(daymet_1.shape)
# daymet_1.tail()

(365, 10)


#### Drop Columns
- year
- dayl
- solar radiation
- snow water equivalent

In [7]:
cols_to_drop = ['year', 'dayl (s)', 'srad (W/m^2)', 'swe (kg/m^2)']

daymet_2 = daymet_1.drop(labels=cols_to_drop, axis=1)
print(daymet_2.shape)
# daymet_2.head()

(365, 6)


#### Rename Columns

In [8]:
daymet_3 = daymet_2.rename({'yday': 'year_of_day', 'prcp (mm/day)': 'precip_mm', 'tmax (deg c)': 'temp_max_c',
                           'tmin (deg c)': 'temp_min_c', 'vp (Pa)': 'vapor_pressure'}, axis=1)

print(daymet_3.shape)
# daymet_3.tail()

(365, 6)


In [9]:
# daymet_3.dtypes

### B. Climate Engine Download
- rh_n - dataset with relative humidity values
- wind_n - dataset with wind values
    - Assumption that these are average wind speed values per day, but have not found the official documentation yet

In [10]:
rh_0 = pd.read_csv('data/raw/clemson_rh_2014.csv')
print(rh_0.shape)
# rh_0.head()

(365, 3)


#### Rename Columns

In [11]:
# rh_0.columns

In [12]:
rh_1 = rh_0.rename({'(%) Min Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'min_rh',
                   '(%) Max Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'max_rh'}, axis=1)

print(rh_1.shape)
# rh_1.head()

(365, 3)


#### Change `DateTime` values to datetime objects

In [13]:
date_range = pd.to_datetime(rh_1.DateTime)
# date_range

In [14]:
rh_2 = rh_1.copy()
rh_2['date'] = date_range
print(rh_2.dtypes)
# rh_2.head()

DateTime            object
min_rh             float64
max_rh             float64
date        datetime64[ns]
dtype: object


#### Drop DateTime Column

In [15]:
rh_3 = rh_2.drop('DateTime', axis=1)
print(rh_3.shape)
# rh_3.tail()

(365, 3)


In [16]:
wind_0 = pd.read_csv('data/raw/climate_engine_wind_speed_2014.csv')
print(wind_0.shape)
# wind_0.head()

(365, 2)


#### Rename Column

In [17]:
wind_1 = wind_0.rename({'(m/s) Wind Speed (gridMET) at -79.7370E,34.2890N, 2014-01-01 to 2014-12-31': 'wind_speed'},
                      axis=1)

print(wind_1.shape)
# wind_1.head()

(365, 2)


#### New Column with datetime object

In [18]:
date_range = pd.to_datetime(wind_1.DateTime)

wind_2 = wind_1.copy()
wind_2['date'] = date_range

print(wind_2.shape)
# wind_2.tail()

(365, 3)


#### Drop DateTime Column

In [19]:
wind_3 = wind_2.drop('DateTime', axis=1)
# wind_3.head()

#### Merge DataFrames

In [20]:
temp_rh_wind_0 = daymet_3.merge(rh_3, how='outer', right_on='date', left_on='date').merge(wind_3, how='outer',
                                                                                    left_on='date', right_on='date')

print(temp_rh_wind_0.shape)
# temp_rh_wind_0.head()

(365, 9)


,year_of_day,precip_mm,temp_max_c,temp_min_c,vapor_pressure,date,min_rh,max_rh,wind_speed
0,1,0,12.0,0.5,640,2014-01-01,39.8632,64.1223,2.1447
1,2,5,10.0,3.0,760,2014-01-02,64.5266,100.0000,1.5169
2,3,11,10.0,-3.5,480,2014-01-03,56.6974,100.0000,7.2649
3,4,0,3.5,-5.5,400,2014-01-04,32.2609,49.2966,3.8728
4,5,0,8.5,-1.5,560,2014-01-05,39.1289,79.4842,2.9018


#### Add cumulative precipitation column

In [22]:
temp_rh_wind_1 = temp_rh_wind_0.copy()

temp_rh_wind_1['cum_precip_mm'] = np.cumsum(temp_rh_wind_1.precip_mm)
print(temp_rh_wind_1.shape)
# temp_rh_wind_1.head(10)

(365, 10)


,year_of_day,precip_mm,temp_max_c,temp_min_c,vapor_pressure,date,min_rh,max_rh,wind_speed,cum_precip_mm
0,1,0,12.0,0.5,640,2014-01-01,39.8632,64.1223,2.1447,0
1,2,5,10.0,3.0,760,2014-01-02,64.5266,100.0000,1.5169,5
2,3,11,10.0,-3.5,480,2014-01-03,56.6974,100.0000,7.2649,16
3,4,0,3.5,-5.5,400,2014-01-04,32.2609,49.2966,3.8728,16
4,5,0,8.5,-1.5,560,2014-01-05,39.1289,79.4842,2.9018,16
5,6,3,16.5,1.0,640,2014-01-06,60.1672,100.0000,6.8690,19
6,7,0,11.0,-10.5,280,2014-01-07,22.2828,41.5916,5.0533,19
7,8,0,0.5,-10.0,280,2014-01-08,8.4978,30.2744,1.7729,19
8,9,0,9.0,-7.0,360,2014-01-09,15.6123,50.9825,2.8636,19
9,10,3,15.0,2.5,720,2014-01-10,33.9721,73.9564,3.6432,22


#### Round values to 2 decimal points

In [24]:
temp_rh_wind_2 = temp_rh_wind_1.copy()

temp_rh_wind_2[['temp_max_c', 'temp_min_c', 'min_rh', 'max_rh', 'wind_speed', 
               'cum_precip_mm']] = temp_rh_wind_2[['temp_max_c', 'temp_min_c', 'min_rh', 'max_rh', 
                                                 'wind_speed', 'cum_precip_mm']].round(2)

print(temp_rh_wind_2.shape)
# temp_rh_wind_2.tail()

(365, 10)


,year_of_day,precip_mm,temp_max_c,temp_min_c,vapor_pressure,date,min_rh,max_rh,wind_speed,cum_precip_mm
360,361,0,17.0,-0.5,600,2014-12-27,25.91,94.88,1.30,1166
361,362,0,19.5,3.0,760,2014-12-28,43.37,75.72,2.12,1166
362,363,2,19.0,11.0,1320,2014-12-29,75.59,100.00,1.92,1168
363,364,1,14.0,5.5,920,2014-12-30,88.34,100.00,4.74,1169
364,365,0,7.5,-1.5,560,2014-12-31,46.47,81.96,3.59,1169


#### Reorder Columns

In [25]:
# temp_rh_wind_2.columns

Index(['year_of_day', 'precip_mm', 'temp_max_c', 'temp_min_c',
       'vapor_pressure', 'date', 'min_rh', 'max_rh', 'wind_speed',
       'cum_precip_mm'],
      dtype='object')

In [26]:
new_col_order = ['date', 'year_of_day', 'temp_min_c', 'temp_max_c', 'min_rh', 'max_rh', 'vapor_pressure', 
                 'wind_speed', 'precip_mm', 'cum_precip_mm']

temp_rh_wind_3 = pd.DataFrame(data=temp_rh_wind_2, columns=new_col_order)
print(temp_rh_wind_3.shape)
temp_rh_wind_3.head(3)

(365, 10)


,date,year_of_day,temp_min_c,temp_max_c,min_rh,max_rh,vapor_pressure,wind_speed,precip_mm,cum_precip_mm
0,2014-01-01,1,0.5,12.0,39.86,64.12,640,2.14,0,0
1,2014-01-02,2,3.0,10.0,64.53,100.00,760,1.52,5,5
2,2014-01-03,3,-3.5,10.0,56.70,100.00,480,7.26,11,16


#### Write to CSV

In [27]:
temp_rh_wind_3.to_csv('data/interim/updated_clemson_weather_2014_2020-06-24.csv', index=False)